In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

<font size=18>Lesson 06 - Self-Assessment Solutions</font>

## *Self Assessment: Solving the California Manufacturing BIP - Solution*

#### Concrete Formulation

In [2]:
# Unfold for code
from pyomo.environ import *

# Concrete Model
model = ConcreteModel(name = "CalMan")

# Decision Variables
model.x = Var( ['factoryLA','factorySF','warehouseLA','warehouseSF'], domain = Boolean)

# Objective 
model.obj = Objective( expr = 9*model.x['factoryLA'] + 5*model.x['factorySF'] + 
                      6*model.x['warehouseLA'] + 4*model.x['warehouseSF'], 
                      sense = maximize)

# Constraints
model.Constraint1 = Constraint( expr = 6*model.x['factoryLA'] + 3*model.x['factorySF'] + 
                      5*model.x['warehouseLA'] + 2*model.x['warehouseSF'] <= 10 )
model.Constraint2 = Constraint( expr = model.x['warehouseLA'] + model.x['warehouseSF'] <= 1 )
model.Constraint3 = Constraint( expr = - model.x['factoryLA'] + model.x['warehouseLA'] <= 0 )
model.Constraint4 = Constraint( expr = - model.x['factorySF'] + model.x['warehouseSF'] <= 0 )

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
print(f"The total net present value is ${model.obj():,.2f} million.")
print(f"Build factory in LA? {['No','Yes'][int(model.x['factoryLA']())]}" )
print(f"Build factory in SF? {['No','Yes'][int(model.x['factorySF']())]}" )
print(f"Build warehouse in LA? {['No','Yes'][int(model.x['warehouseLA']())]}" )
print(f"Build warehouse in SF? {['No','Yes'][int(model.x['warehouseLA']())]}" )

The total net present value is $14.00 million.
Build factory in LA? Yes
Build factory in SF? Yes
Build warehouse in LA? No
Build warehouse in SF? No


#### Abstract Formulation

In [3]:
# Unfold for code
from pyomo.environ import *
decisions = ['factoryLA', 'factorySF', 'warehouseLA', 'warehouseSF']
npv = dict(zip(decisions, [9, 5, 6, 4]))
cap = dict(zip(decisions, [6, 3, 5, 2]))

total_capital = 10

#Concrete Model
model = ConcreteModel()

#Decision Variables
model.build = Var(decisions, domain=Boolean)

#Objective
model.npv = Objective(expr=sum(npv[d] * model.build[d]
                                  for d in decisions),
                         sense=maximize)

cts = ['ct1','ct2','ct3']
coefs = {'ct1':dict(zip(decisions,[ 0,0,1,1])),
         'ct2':dict(zip(decisions,[-1,0,1,0])),
         'ct3':dict(zip(decisions,[0,-1,0,1]))}
rhs = dict(zip(cts,[1,0,0]))

model.constraints = ConstraintList()
model.constraints.add(sum(cap[d] * model.build[d] for d in decisions) <= total_capital)
for c in cts:
    model.constraints.add( expr = sum(coefs[c][d]*model.build[d] for d in decisions) <= rhs[c])

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
print(f"The total net present value is ${model.npv():,.2f} million.")
for d in decisions:
    print(f"Build {d}? {['No','Yes'][int(model.build[d]())]}" )


The total net present value is $14.00 million.
Build factoryLA? Yes
Build factorySF? Yes
Build warehouseLA? No
Build warehouseSF? No


## *Self-Assessment: Integer Programming - Solution*

Answer:  True

## *Self-Assessment: Type of Programming - Solution*

Answer:  c

## *Self-Assessment: Rounding Solutions to Integers - Solution*

Answer: e (read section 12.5 again if you missed this)

## *Self-Assessment: Either/Or Constraints - Solution*

Answer: False (only one binary variable is needed)

## *Self-Assessment: Number of Solutions in BIP - Solution*

Answer: False (there are $2^n$ solutions, the number of solutions grows exponentially with the number of variables, see 12.5)